In [1]:
import serial
import time       
import io
from nesp_lib import Port, Pump, PumpingDirection

class CellWashing:
    
    # Start on class instantiate
    def __init__(self, valve_com_port, syringe_com_port):
        
        self.valve_com_port = valve_com_port
        self.syringe_com_port = syringe_com_port
        
        # Connect valve via the specified com_port
        self.ser = serial.Serial(port = self.valve_com_port,  #Check your device manager and change the COM port here as needed.
                    baudrate = 9600, 
                    bytesize = serial.EIGHTBITS,  
                    parity = serial.PARITY_NONE,   #Establishes a serial connection between valve controller and computer.
                    stopbits = serial.STOPBITS_ONE
                   )
        
        print('Valve Connected')
        
        # Connect syringe via the specified com_port
        self.port = Port(self.syringe_com_port)
        self.pump = Pump(self.port, address = 1)
        time.sleep(1)
        
        # Sets the syringe diameter of the pump in units of millimeters.
        self.pump.syringe_diameter = 26.64
        
        # Sets the pumping rate of the pump in units of milliliters per minute.
        self.pump.pumping_rate = 25
        
        print('Syringe Pump Connected')
    
    # This is for the serial communications with the valve
    def _readline(self): #Used to ensure readline() end at \r since readline() function no longer exists in Python 3.x
        
        eol = b'\r'
        leneol = len(eol)
        line = bytearray()
        while True:
            c = self.ser.read(1)
            if c:
                line += c
                if line[-leneol:] == eol:
                    break
            else:
                break
        print(bytes(line))
        return bytes(line)

    # Def move the valve to any direction that is wished
    def move_valve(self, valve_num):
        
        # Input check
        if valve_num not in [1,2,3,4,5,6]:
            raise NameError(f'Invalid valve change position - must be [1,2,3,4,5,6]')
                
        print(f'Valve request to moved to position {valve_num}')
        position = f'GO0{valve_num}\r'     
        self.ser.write(position.encode()) #Serial command communicates and "instructs" the valve controller to change positions.
        time.sleep(1)

        self.ser.write(b'CP\r')     #This serial command prints the current position of the valve.
        time.sleep(0.2)
        self._readline()
    
    # Definitions of the pump
    # Port 1 - DI Water Source
    # Port 2 - Deposition Cell
    # Port 3 - Characterization Cell
    # Port 4 - Waste
    
    # The following code is purely for the cell washing steps
    def wash_deposition_cell(self, volume, num_wash_steps): 
        
        # Sets the pumping volume of the pump in units of milliliters.
        self.pump.pumping_volume = volume
        
        # Initial withdraw of liquid from cell
        self.move_valve(2)
        self.pump.pumping_direction = PumpingDirection.WITHDRAW
        self.pump.run()
        self.move_valve(4)
        self.pump.pumping_direction = PumpingDirection.INFUSE
        self.pump.run()
        
        # Washing loop 
        for i in range(num_wash_steps):
                self.move_valve(1)
                self.pump.pumping_direction = PumpingDirection.WITHDRAW
                self.pump.run()
                self.move_valve(2)
                self.pump.pumping_direction = PumpingDirection.INFUSE
                self.pump.run()
                self.pump.pumping_direction = PumpingDirection.WITHDRAW
                self.pump.run()
                self.move_valve(4)
                self.pump.pumping_direction = PumpingDirection.INFUSE
                self.pump.run()
                
    def wash_characterization_cell(self, volume, num_wash_steps): 
        
        # Sets the pumping volume of the pump in units of milliliters.
        self.pump.pumping_volume = volume
        
        # Initial withdraw of liquid from cell
        self.move_valve(3)
        self.pump.pumping_direction = PumpingDirection.WITHDRAW
        self.pump.run()
        self.move_valve(4)
        self.pump.pumping_direction = PumpingDirection.INFUSE
        self.pump.run()
        
        # Washing loop 
        for i in range(num_wash_steps):
                self.move_valve(1)
                self.pump.pumping_direction = PumpingDirection.WITHDRAW
                self.pump.run()
                self.move_valve(3)
                self.pump.pumping_direction = PumpingDirection.INFUSE
                self.pump.run()
                self.pump.pumping_direction = PumpingDirection.WITHDRAW
                self.pump.run()
                self.move_valve(4)
                self.pump.pumping_direction = PumpingDirection.INFUSE
                self.pump.run()
        

In [2]:
syringe = CellWashing(valve_com_port= 'COM18', syringe_com_port= 'COM13')

Valve Connected
Syringe Pump Connected


In [5]:
syringe.wash_characterization_cell(volume  = 30, num_wash_steps = 3)

Valve request to moved to position 3
b'CP03\r'
Valve request to moved to position 4
b'CP04\r'
Valve request to moved to position 1
b'CP01\r'
Valve request to moved to position 3
b'CP03\r'
Valve request to moved to position 4
b'CP04\r'
Valve request to moved to position 1
b'CP01\r'
Valve request to moved to position 3
b'CP03\r'
Valve request to moved to position 4
b'CP04\r'
Valve request to moved to position 1
b'CP01\r'
Valve request to moved to position 3
b'CP03\r'


SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [2]:
syringe_pump = SyringePump('COM18')

Valve Connected


In [4]:
syringe_pump.move_valve(1)

Valve request to moved to position 1
b'CP01\r'


***Fiddle code for the new era syringe pump***

In [1]:
import nesp_lib

In [1]:
from nesp_lib import Port, Pump, PumpingDirection

In [2]:
port = Port('COM13')
pump = Pump(port, address = 2)

In [3]:
# Prints the model number of the pump (e.g. "1000" for NE-1000).
print(pump.model_number)
# Prints the firmware version of the pump (e.g. "(3, 928)" for 3.928).
print(pump.firmware_version)

1000
(3, 934)


In [7]:
# Sets the syringe diameter of the pump in units of millimeters.
pump.syringe_diameter = 26.64
# Sets the pumping direction of the pump.
pump.pumping_direction = PumpingDirection.INFUSE
# Sets the pumping volume of the pump in units of milliliters.
pump.pumping_volume = 10
# Sets the pumping rate of the pump in units of milliliters per minute.
pump.pumping_rate = 25

In [8]:
# Runs the pump considering the direction, volume, and rate set.
pump.run()

In [10]:
# Sets the syringe diameter of the pump in units of millimeters.
pump.syringe_diameter = 30.0

***Fiddle code for the valvo VICI valve connection***

In [2]:
import serial
import time       
import io

ser = serial.Serial(port = 'COM18',  #Check your device manager and change the COM port here as needed.
                    baudrate = 9600, 
                    bytesize = serial.EIGHTBITS,  
                    parity = serial.PARITY_NONE,   #Establishes a serial connection between valve controller and computer.
                    stopbits = serial.STOPBITS_ONE
                   )

In [3]:
def odd_move(): 
    move = input("Please enter the position which you want to move to.") 
    allowed = ["1","2","3","4","5","6"]   
    if move not in allowed:                            #Code checks that the user's input is one of the allowed inputs.
        print("You must select either 1,3,5,7, or 9!")            
    elif move in allowed:
         int(move)
         position = f'GO{move}\r'     #Used f string, and serial "GO" command to move valve into user's desired position.
         ser.write(position.encode()) #Serial command communicates and "instructs" the valve controller to change positions.
         time.sleep(1)
        
         ser.write(b'CP\r')     #This serial command prints the current position of the valve.
         time.sleep(0.2)
         _readline()
            
def _readline(): #Used to ensure readline() end at \r since readline() function no longer exists in Python 3.x
    eol = b'\r'
    leneol = len(eol)
    line = bytearray()
    while True:
        c = ser.read(1)
        if c:
            line += c
            if line[-leneol:] == eol:
                break
        else:
            break
    print(bytes(line))
    return bytes(line)

In [5]:
 position = f'GO04\r'     #Used f string, and serial "GO" command to move valve into user's desired position.
 ser.write(position.encode()) #Serial command communicates and "instructs" the valve controller to change positions.
 time.sleep(1)

 ser.write(b'CP\r')     #This serial command prints the current position of the valve.
 time.sleep(0.2)
 _readline()

b'CP04\r'


b'CP04\r'